***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Intro: Parsing als automatische Verarbeitung formaler Grammatiken
- **formale Grammatik**: Modellierung der syntaktischen Struktur (eines Ausschnitts) natürlicher Sprache 
  - ermöglicht Generierung der wohlgeformten Sätze der Sprache
  - durch wiederholte Regelanwendung und Rekursion: beliebig lange Sätze möglich 
- **Parsing-Algorithmus**: Verfahren zur Überprüfung einer Eingabe auf Wohlgeformtheit gemäß einer Grammatik

In [1]:
import nltk

In [2]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    PP -> P NP
    NP -> Det N
    NP -> NP PP
    NP -> Pron
    VP -> V NP | VP PP
##Lexikalische Regeln:
    Pron -> 'I'      
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")


## Generierung mit formaler Grammatik:

In [3]:
#Generierung (depth = maximale Anzahl an Regelanwendungen):
from nltk.parse.generate import generate
for sentence in generate(grammar, depth=5):
    print(' '.join(sentence))

an elephant shot an elephant
an elephant shot an pajamas
an elephant shot my elephant
an elephant shot my pajamas
an elephant shot I
an pajamas shot an elephant
an pajamas shot an pajamas
an pajamas shot my elephant
an pajamas shot my pajamas
an pajamas shot I
my elephant shot an elephant
my elephant shot an pajamas
my elephant shot my elephant
my elephant shot my pajamas
my elephant shot I
my pajamas shot an elephant
my pajamas shot an pajamas
my pajamas shot my elephant
my pajamas shot my pajamas
my pajamas shot I
I shot an elephant
I shot an pajamas
I shot my elephant
I shot my pajamas
I shot I


## Syntaktische Ambiguität

In [4]:
sent = 'I shot an elephant in my pajamas'.split()
print(sent)

['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']


In [5]:
parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (Pron I))
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
      S                                       
 ┌────┴──────────────┐                         
 │                   VP                       
 │         ┌─────────┴──────────┐              
 │         VP                   PP            
 │    ┌────┴───┐            ┌───┴───┐          
 NP   │        NP           │       NP        
 │    │    ┌───┴─────┐      │   ┌───┴─────┐    
Pron  V   Det        N      P  Det        N   
 │    │    │         │      │   │         │    
 I   shot  an     elephant  in  my     pajamas

(S
  (NP (Pron I))
  (VP
    (V shot)
    (NP
      (NP (Det an) (N elephant))
      (PP (P in) (NP (Det my) (N pajamas))))))
      S                                       
 ┌────┴──────────────┐                         
 │                   VP                       
 │    ┌──────────────┴──────┐                  
 │    │                     NP                


## Koordinationsambiguität

In [6]:
grammar = nltk.CFG.fromstring("""
## linke Seite erster Regel = Startsymbol:
    NP -> NP Con NP
    NP -> Adj NP
    NP -> N
    Con -> 'und'
    Adj -> 'alte'
    N -> 'Männer' | 'Frauen'
""")

sent = 'alte Männer und Frauen'.split()

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(NP (NP (Adj alte) (NP (N Männer))) (Con und) (NP (N Frauen)))
           NP             
      ┌────┴─────┬────┐    
      NP         │    │   
 ┌────┴────┐     │    │    
 │         NP    │    NP  
 │         │     │    │    
Adj        N    Con   N   
 │         │     │    │    
alte     Männer und Frauen

(NP (Adj alte) (NP (NP (N Männer)) (Con und) (NP (N Frauen))))
             NP           
 ┌───────────┴───┐         
 │               NP       
 │     ┌─────────┼────┐    
 │     NP        │    NP  
 │     │         │    │    
Adj    N        Con   N   
 │     │         │    │    
alte Männer     und Frauen



## Koordination: nur gleiche Phrasen!

In [7]:
#Grammatik ohne Einschränkung der Koordination auf gleichartige Phrasen:
grammar = nltk.CFG.fromstring("""
    XP -> XP Con XP
    XP -> NP | ADJP
    NP -> N
    ADJP -> Adj
    Con -> 'und'
    N -> 'Hund' | 'Katze' | 'Esel'
    Adj -> 'groß' | 'klein'
""")

#Generierung:
from nltk.parse.generate import generate
for sentence in generate(grammar, depth=5):
    print(' '.join(sentence))

Hund und Hund
Hund und Katze
Hund und Esel
Hund und groß
Hund und klein
Katze und Hund
Katze und Katze
Katze und Esel
Katze und groß
Katze und klein
Esel und Hund
Esel und Katze
Esel und Esel
Esel und groß
Esel und klein
groß und Hund
groß und Katze
groß und Esel
groß und groß
groß und klein
klein und Hund
klein und Katze
klein und Esel
klein und groß
klein und klein
Hund
Katze
Esel
groß
klein



## Temporale Ambiguität (Backtracking)

In [8]:
sent = 'the old man the boat'.split()

grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    NP -> Det Adj N
    NP -> Det N
    VP -> V NP
##Lexikalische Regeln:
    Det -> 'the'
    Adj -> 'old'    
    N -> 'man' | 'boat' | 'old'
    V -> 'man'
""")


In [9]:
#NLTK-ShiftReduceParser: kein Backtracking! 
#bleibt bei Analyse NP NP ((the old man) (the boat)) stehen, findet keinen vollständigen Parse

parser = nltk.ShiftReduceParser(grammar,trace=2)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)
    

Parsing 'the old man the boat'
    [ * the old man the boat]
  S [ 'the' * old man the boat]
  R [ Det * old man the boat]
  S [ Det 'old' * man the boat]
  R [ Det Adj * man the boat]
  S [ Det Adj 'man' * the boat]
  R [ Det Adj N * the boat]
  R [ NP * the boat]
  S [ NP 'the' * boat]
  R [ NP Det * boat]
  S [ NP Det 'boat' * ]
  R [ NP Det N * ]
  R [ NP NP * ]


In [10]:
##Reanalyse der Struktur durch Backtracking mit RecursiveDescentParser:

parser = nltk.RecursiveDescentParser(grammar,trace=3)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)
    

Parsing 'the old man the boat'
Start:
    [ * S ]
Expand: S -> NP VP
    [ * NP VP ]
Expand: NP -> Det Adj N
    [ * Det Adj N VP ]
Expand: Det -> 'the'
    [ * 'the' Adj N VP ]
Match: 'the'
    [ 'the' * Adj N VP ]
Expand: Adj -> 'old'
    [ 'the' * 'old' N VP ]
Match: 'old'
    [ 'the' 'old' * N VP ]
Expand: N -> 'man'
    [ 'the' 'old' * 'man' VP ]
Match: 'man'
    [ 'the' 'old' 'man' * VP ]
Expand: VP -> V NP
    [ 'the' 'old' 'man' * V NP ]
Expand: V -> 'man'
    [ 'the' 'old' 'man' * 'man' NP ]
Backtrack: 'the' match failed
Expand: N -> 'boat'
    [ 'the' 'old' * 'boat' VP ]
Backtrack: 'man' match failed
Expand: N -> 'old'
    [ 'the' 'old' * 'old' VP ]
Backtrack: 'man' match failed
Expand: NP -> Det N
    [ * Det N VP ]
Expand: Det -> 'the'
    [ * 'the' N VP ]
Match: 'the'
    [ 'the' * N VP ]
Expand: N -> 'man'
    [ 'the' * 'man' VP ]
Backtrack: 'old' match failed
Expand: N -> 'boat'
    [ 'the' * 'boat' VP ]
Backtrack: 'old' match failed
Expand: N -> 'old'
    [ 'the' * 'old

---

---


## Verarbeitung von CFGs mit verschiedenen Parsern

- Recursive-Descent-Parser kann keine linksrekursiven Regeln verarbeiten!

In [11]:
sent = 'I shot an elephant'.split()
print(sent)

['I', 'shot', 'an', 'elephant']


In [12]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    PP -> P NP
    NP -> Det N 
#rekursive Regel:
    NP -> NP PP
    NP -> Pron
    VP -> V NP
#rekursive Regel:    
    VP -> VP PP
##Lexikalische Regeln:
    Pron -> 'I'      
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")



### Shift-Reduce-Parser (bottom-up)
- https://www.nltk.org/book/ch08.html#shift-reduce-parsing

In [13]:
parser = nltk.ShiftReduceParser(grammar,trace=2)
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print()

Parsing 'I shot an elephant'
    [ * I shot an elephant]
  S [ 'I' * shot an elephant]
  R [ Pron * shot an elephant]
  R [ NP * shot an elephant]
  S [ NP 'shot' * an elephant]
  R [ NP V * an elephant]
  S [ NP V 'an' * elephant]
  R [ NP V Det * elephant]
  S [ NP V Det 'elephant' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP (Pron I)) (VP (V shot) (NP (Det an) (N elephant))))
           S                  
  _________|___                
 |             VP             
 |     ________|___            
 NP   |            NP         
 |    |         ___|_____      
Pron  V       Det        N    
 |    |        |         |     
 I   shot      an     elephant



### Chart-Parser (bottom-up)
- https://www.nltk.org/book/ch08.html#well-formed-substring-tables

In [14]:
parser = nltk.ChartParser(grammar,trace=1)
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print()

|.    I    .   shot  .    an   . elephant.|
|[---------]         .         .         .| [0:1] 'I'
|.         [---------]         .         .| [1:2] 'shot'
|.         .         [---------]         .| [2:3] 'an'
|.         .         .         [---------]| [3:4] 'elephant'
|[---------]         .         .         .| [0:1] Pron -> 'I' *
|[---------]         .         .         .| [0:1] NP -> Pron *
|[--------->         .         .         .| [0:1] S  -> NP * VP
|[--------->         .         .         .| [0:1] NP -> NP * PP
|.         [---------]         .         .| [1:2] V  -> 'shot' *
|.         [--------->         .         .| [1:2] VP -> V * NP
|.         .         [---------]         .| [2:3] Det -> 'an' *
|.         .         [--------->         .| [2:3] NP -> Det * N
|.         .         .         [---------]| [3:4] N  -> 'elephant' *
|.         .         [-------------------]| [2:4] NP -> Det N *
|.         .         [------------------->| [2:4] S  -> NP * VP
|.         .         

### Recursive-Descent-Parser (top-down)
- https://www.nltk.org/book/ch08.html#recursive-descent-parsing

- Endlosschleife bei linksrekursiver Regel!

In [15]:
#parser = nltk.RecursiveDescentParser(grammar,trace=2)
#for tree in parser.parse(sent):
#    print(tree)
#    tree.pretty_print()

```
Parsing 'I shot an elephant'
    [ * S ]
  E [ * NP VP ]
  E [ * Det N VP ]
  E [ * 'an' N VP ]
  E [ * 'my' N VP ]
  E [ * NP PP VP ]
  E [ * Det N PP VP ]
  E [ * 'an' N PP VP ]
  E [ * 'my' N PP VP ]
  E [ * NP PP PP VP ]
  E [ * Det N PP PP VP ]
  E [ * 'an' N PP PP VP ]
  E [ * 'my' N PP PP VP ]
  E [ * NP PP PP PP VP ]
  E [ * Det N PP PP PP VP ]
  E [ * 'an' N PP PP PP VP ]
  E [ * 'my' N PP PP PP VP ]
  E [ * NP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP PP PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP PP PP PP PP PP PP PP VP ]
```

---

---

# Dependenzgrammatik: Verarbeitung nicht-projektiver Strukturen

- (Dependenzgrammatik im NLTK: https://www.nltk.org/book/ch08.html#dependencies-and-dependency-grammar)

- nicht-projektive (diskontinuierliche) Strukturen können mit Dependenzgrammatiken modelliert werden, da sie (im Gegensatz zu Konstituentengrammatiken) von der linearen Anordnung abstrahieren (Modellierung der Abhängigkeitsbeziehungen)
- allerdings können in der Verarbeitung von Dependenzgrammatiken (Dependency Parsing) solche Strukturen Probleme bereiten
- vgl. https://en.wikipedia.org/wiki/Discontinuity_(linguistics)#Projectivity

In [16]:
grammar = nltk.DependencyGrammar.fromstring("""
    'Hund' -> 'ein' | ','
    'verstummte' -> 'Hund'
    'Hund' -> 'gebellt'
    'gebellt' -> 'der' | 'vorher' | 'hatte'
    'laut' -> 'sehr'
    """)

### projektive Struktur: *Relativsatz*

In [17]:
sent = 'ein Hund , der vorher gebellt hatte , verstummte'.split()
print(sent)

['ein', 'Hund', ',', 'der', 'vorher', 'gebellt', 'hatte', ',', 'verstummte']


In [18]:
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(verstummte (Hund ein , (gebellt der vorher hatte) ,))
        verstummte                  
            │                        
           Hund                     
 ┌───┬──────┼─────────────┐          
 │   │      │          gebellt      
 │   │      │       ┌─────┼──────┐   
ein  ,      ,      der  vorher hatte



### nicht-projektive Struktur (*long distance dependency*): *ins Nachfeld extrahierter Relativsatz*
- *verstummte* wird in die komplexe Nominalphrase eingeschoben
- Überschneidung von Kanten: der Dependent (*gebellt*) von *Hund* folgt nach dessen Kopf (*verstummte*) 
- (Details dazu in Sitzung 11)


In [19]:
sent = 'ein Hund verstummte , der vorher gebellt hatte'.split()
print(sent)

['ein', 'Hund', 'verstummte', ',', 'der', 'vorher', 'gebellt', 'hatte']


In [20]:
#kein Parse mit ProjectiveDependencyParser!
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

In [21]:
#NonprojectiveDependencyParser: erkennt Struktur korrekt
#Code s. http://www.nltk.org/howto/dependency.html:

dp = nltk.NonprojectiveDependencyParser(grammar)
g, = dp.parse(sent)

print(g.root['word'])

for _, node in sorted(g.nodes.items()):
    if node['word'] is not None:
        print('{address} {word}: {d}'.format(d=node['deps'][''], **node))

print('\n', g.tree(), '\n')
g.tree().pretty_print(unicodelines=True)

verstummte
1 ein: []
2 Hund: [1, 4, 7]
3 verstummte: [2]
4 ,: []
5 der: []
6 vorher: []
7 gebellt: [5, 6, 8]
8 hatte: []

 (verstummte (Hund ein , (gebellt der vorher hatte))) 

        verstummte              
            │                    
           Hund                 
 ┌───┬──────┴─────────┐          
 │   │             gebellt      
 │   │      ┌─────────┼──────┐   
ein  ,     der      vorher hatte

